# Notebook 02 - Feature Engineering for CLD (Early-passages only)

## Goal
Convert the raw CLD relational tables (passage-level assay measurements) into a **clone-level ML dataset**.

We will:
1. Load 'assay_result' joined with 'passage' from SQLite
2. Restrict to **early passages** (default: 1-5)
3. Create clone-level features (X), such as:
    - early mean titer / VCD / viability / aggregation
    - early slope (trend) of titer over passages
    - early variability (std) across passages
4. Join with the stability label (y) from 'stability_test'
5. Save a ML-ready feature table

## Why this matters
In real CLD, we must decide which clones to advance **using early data only**.
This notebook creates. the dataset needed to train a model for early clone selection.

## 1) Imports and database connection

In [15]:
import sqlite3
import pandas as pd
import numpy as np
import os

DB_PATH = "../data/synthetic/raw/cld.db"
conn = sqlite3.connect(DB_PATH)
print("Connected to:", DB_PATH)

Connected to: ../data/synthetic/raw/cld.db


## 2) Load assay results joined with passage metadata

We JOIN 'assay_result' with 'passage' so each measurement includes:
- clone_id
- passage_number
- phase (early/mid/late)

In [16]:
assay = pd.read_sql_query("""
SELECT 
  ar.assay_id,
  ar.assay_type,
  ar.value,
  ar.unit,
  ar.method,
  ar.batch_id,
  p.clone_id,
  p.passage_number,
  p.phase
FROM assay_result ar
JOIN passage p
  ON p.passage_id = ar.passage_id
""", conn)

assay.head()

,assay_id,assay_type,value,unit,method,batch_id,clone_id,passage_number,phase
0,ASSAY_CLONE_0001_P01_titer,titer,2.865790e+00,g/L,ELISA,B_P01,CLONE_0001,1,early
1,ASSAY_CLONE_0001_P01_vcd,vcd,9.082159e+06,cells/mL,Vi-CELL,B_P01,CLONE_0001,1,early
2,ASSAY_CLONE_0001_P01_viability,viability,9.379353e+01,%,Vi-CELL,B_P01,CLONE_0001,1,early
3,ASSAY_CLONE_0001_P01_aggregation,aggregation,8.054375e+00,%,SEC-HPLC,B_P01,CLONE_0001,1,early
4,ASSAY_CLONE_0001_P02_titer,titer,2.800113e+00,g/L,ELISA,B_P02,CLONE_0001,2,early


## 3) Restrict to early passages

We build features using early passages only.
Default window: passages 1-5.

This is critical to avoid data leakage and mimic real real CLD screening.

In [17]:
EARLY_START = 3
EARLY_END = 10

assay_early = assay[(assay["passage_number"] >= EARLY_START) & (assay["passage_number"] <= EARLY_END)].copy()

print("Rows in assay (all):", len(assay))
print("Rows in assay (early):", len(assay_early))
assay_early.head()

Rows in assay (all): 60000
Rows in assay (early): 16000


,assay_id,assay_type,value,unit,method,batch_id,clone_id,passage_number,phase
8,ASSAY_CLONE_0001_P03_titer,titer,2.964514e+00,g/L,ELISA,B_P03,CLONE_0001,3,early
9,ASSAY_CLONE_0001_P03_vcd,vcd,9.815405e+06,cells/mL,Vi-CELL,B_P03,CLONE_0001,3,early
10,ASSAY_CLONE_0001_P03_viability,viability,9.342190e+01,%,Vi-CELL,B_P03,CLONE_0001,3,early
11,ASSAY_CLONE_0001_P03_aggregation,aggregation,8.451790e+00,%,SEC-HPLC,B_P03,CLONE_0001,3,early
12,ASSAY_CLONE_0001_P04_titer,titer,2.799705e+00,g/L,ELISA,B_P04,CLONE_0001,4,early


## 4) Pivot early assay data into a wide table (one row per clone per passage)

Raw data is "long format" (one row per assay measurement).
For many feature calculations, it is convenient to pivot into "wide format":

Colums will become:
- titer, vcd, viability, aggregation

In [18]:
early_wide = assay_early.pivot_table(
    index=["clone_id", "passage_number"],
    columns="assay_type",
    values="value",
    aggfunc="mean"
).reset_index()

early_wide.head()

assay_type,clone_id,passage_number,aggregation,titer,vcd,viability
0,CLONE_0001,3,8.451790,2.964514,9.815405e+06,93.421901
1,CLONE_0001,4,8.056273,2.799705,1.197343e+07,94.073862
2,CLONE_0001,5,8.136477,2.679678,1.136983e+07,94.372907
3,CLONE_0001,6,8.488188,2.721633,9.779103e+06,97.386709
4,CLONE_0001,7,8.894030,2.226854,1.117354e+07,95.478145


## 5) Compute clone-level early summary features

For each clone, we aggregate early passage values into summary features:
- mean
- std (variability)
- min/max

These capture early productivity, growth, health, and quality signals.

In [19]:
metrics = ["titer", "vcd", "viability", "aggregation"]

agg_dict = {}
for m in metrics:
    agg_dict[m] = ["mean", "std", "min", "max"]

summary = early_wide.groupby("clone_id")[metrics].agg(agg_dict)

# Flatten multi-index column names, e.g., titer_mean, vcd_std, ...
summary.columns = [f"{col[0]}_{col[1]}" for col in summary.columns]
summary = summary.reset_index()

summary.head()

,clone_id,titer_mean,titer_std,titer_min,titer_max,vcd_mean,vcd_std,vcd_min,vcd_max,viability_mean,viability_std,viability_min,viability_max,aggregation_mean,aggregation_std,aggregation_min,aggregation_max
0,CLONE_0001,2.538067,0.293036,2.223711,2.964514,1.107619e+07,9.052556e+05,9.779103e+06,1.232258e+07,94.851455,1.271241,93.421901,97.386709,8.356812,0.403848,7.723455,8.894030
1,CLONE_0002,0.814721,0.213007,0.537981,1.132518,1.441091e+07,1.047019e+06,1.346070e+07,1.605341e+07,97.551824,1.348922,95.753454,99.573812,7.337188,0.449925,6.798531,8.011004
2,CLONE_0003,3.912552,0.208697,3.621956,4.261524,8.684126e+06,5.834120e+05,7.780120e+06,9.384241e+06,94.390688,2.607492,90.736348,98.774683,2.235352,0.326064,1.745532,2.861836
3,CLONE_0004,0.488369,0.160312,0.212916,0.747609,1.511725e+07,7.817665e+05,1.405453e+07,1.606425e+07,96.380534,1.730106,94.237331,98.968096,3.860240,0.305128,3.357574,4.286918
4,CLONE_0005,2.238289,0.160672,2.033612,2.459557,1.117195e+07,1.167196e+06,8.874346e+06,1.239030e+07,95.085238,1.326355,93.026315,96.870619,3.231717,0.405157,2.620058,4.001488


## 6) Compute early slope features (trend over passages)

In CLD, trend can matter:
- A clone with early titer decreasing quickly may be less stable.
- A clone with improving viability may be adapting well.

We compute the slope of each metric vs passage_number using a simple linear fit.

In [20]:
def slope(x, y):
    """Return slope of y ~ a*x + b. Uses least squares. Handles small N."""
    if len(x) < 2:
        return np.nan
    return np.polyfit(x, y, 1)[0]

slope_rows = []
for clone_id, df in early_wide.groupby("clone_id"):
    x = df["passage_number"].values
    row = {"clone_id": clone_id}
    for m in metrics:
        if m in df.columns:
            y = df[m].values
            row[f"{m}_slope"] = slope(x, y)
        else:
            row[f"{m}_slope"] = np.nan
    slope_rows.append(row)

slopes = pd.DataFrame(slope_rows)
slopes.head()

,clone_id,titer_slope,vcd_slope,viability_slope,aggregation_slope
0,CLONE_0001,-0.103703,102268.665747,0.169734,-0.019225
1,CLONE_0002,-0.005035,279733.381794,0.198861,-0.023819
2,CLONE_0003,-0.058117,51853.805966,0.505344,0.038583
3,CLONE_0004,-0.026671,278866.456374,0.504642,0.051834
4,CLONE_0005,-0.034059,391064.583721,0.233446,-0.129212


## 7) Merge all features into a single clone-level feature table

We combine:
- early summary features (mean/std/min/max)
- early slope features

In [21]:
X = summary.merge(slopes, on="clone_id", how="left")

print("Feature table shape:", X.shape)
X.head()

Feature table shape: (500, 21)


,clone_id,titer_mean,titer_std,titer_min,titer_max,vcd_mean,vcd_std,vcd_min,vcd_max,viability_mean,...,viability_min,viability_max,aggregation_mean,aggregation_std,aggregation_min,aggregation_max,titer_slope,vcd_slope,viability_slope,aggregation_slope
0,CLONE_0001,2.538067,0.293036,2.223711,2.964514,1.107619e+07,9.052556e+05,9.779103e+06,1.232258e+07,94.851455,...,93.421901,97.386709,8.356812,0.403848,7.723455,8.894030,-0.103703,102268.665747,0.169734,-0.019225
1,CLONE_0002,0.814721,0.213007,0.537981,1.132518,1.441091e+07,1.047019e+06,1.346070e+07,1.605341e+07,97.551824,...,95.753454,99.573812,7.337188,0.449925,6.798531,8.011004,-0.005035,279733.381794,0.198861,-0.023819
2,CLONE_0003,3.912552,0.208697,3.621956,4.261524,8.684126e+06,5.834120e+05,7.780120e+06,9.384241e+06,94.390688,...,90.736348,98.774683,2.235352,0.326064,1.745532,2.861836,-0.058117,51853.805966,0.505344,0.038583
3,CLONE_0004,0.488369,0.160312,0.212916,0.747609,1.511725e+07,7.817665e+05,1.405453e+07,1.606425e+07,96.380534,...,94.237331,98.968096,3.860240,0.305128,3.357574,4.286918,-0.026671,278866.456374,0.504642,0.051834
4,CLONE_0005,2.238289,0.160672,2.033612,2.459557,1.117195e+07,1.167196e+06,8.874346e+06,1.239030e+07,95.085238,...,93.026315,96.870619,3.231717,0.405157,2.620058,4.001488,-0.034059,391064.583721,0.233446,-0.129212


## 8) Load stability labels (target y) and merge with features

The target labels is stored in 'stability_test.productivity_drop_pct'. We merge it with the feature table to create the ML dataset (X,y).

In [22]:
y = pd.read_sql_query("""
SELECT clone_id, productivity_drop_pct
FROM stability_test
""", conn)

dataset = X.merge(y, on="clone_id", how="inner")

print("Dataset shape (X + y):", dataset.shape)
dataset.head()

Dataset shape (X + y): (500, 22)


,clone_id,titer_mean,titer_std,titer_min,titer_max,vcd_mean,vcd_std,vcd_min,vcd_max,viability_mean,...,viability_max,aggregation_mean,aggregation_std,aggregation_min,aggregation_max,titer_slope,vcd_slope,viability_slope,aggregation_slope,productivity_drop_pct
0,CLONE_0001,2.538067,0.293036,2.223711,2.964514,1.107619e+07,9.052556e+05,9.779103e+06,1.232258e+07,94.851455,...,97.386709,8.356812,0.403848,7.723455,8.894030,-0.103703,102268.665747,0.169734,-0.019225,0.387063
1,CLONE_0002,0.814721,0.213007,0.537981,1.132518,1.441091e+07,1.047019e+06,1.346070e+07,1.605341e+07,97.551824,...,99.573812,7.337188,0.449925,6.798531,8.011004,-0.005035,279733.381794,0.198861,-0.023819,0.135156
2,CLONE_0003,3.912552,0.208697,3.621956,4.261524,8.684126e+06,5.834120e+05,7.780120e+06,9.384241e+06,94.390688,...,98.774683,2.235352,0.326064,1.745532,2.861836,-0.058117,51853.805966,0.505344,0.038583,0.335258
3,CLONE_0004,0.488369,0.160312,0.212916,0.747609,1.511725e+07,7.817665e+05,1.405453e+07,1.606425e+07,96.380534,...,98.968096,3.860240,0.305128,3.357574,4.286918,-0.026671,278866.456374,0.504642,0.051834,0.590633
4,CLONE_0005,2.238289,0.160672,2.033612,2.459557,1.117195e+07,1.167196e+06,8.874346e+06,1.239030e+07,95.085238,...,96.870619,3.231717,0.405157,2.620058,4.001488,-0.034059,391064.583721,0.233446,-0.129212,0.291757


## 9) Sanity checks sfor ML readiness

We check:
- missing values
- basic distribution of the target label

In [23]:
missing = dataset.isna().mean().sort_values(ascending=False).head(10)
print("Top missing-rate columns:")
print(missing)

print("\nTarget label summary:")
print(dataset["productivity_drop_pct"].describe())

Top missing-rate columns:
clone_id             0.0
titer_mean           0.0
aggregation_slope    0.0
viability_slope      0.0
vcd_slope            0.0
titer_slope          0.0
aggregation_max      0.0
aggregation_min      0.0
aggregation_std      0.0
aggregation_mean     0.0
dtype: float64

Target label summary:
count    500.000000
mean       0.268639
std        0.131551
min       -0.333729
25%        0.178165
50%        0.270368
75%        0.355440
max        0.755038
Name: productivity_drop_pct, dtype: float64


## 10) Save outputs for modeling

We save:
- the ML-ready dataset (feature + label) as CSV
- optionally, features-only rable

In [24]:
OUT_PATH = "../data/synthetic/processed"
os.makedirs(OUT_PATH, exist_ok=True)

dataset.to_csv(f"{OUT_PATH}/cld_features_with_label.csv", index=False)
X.to_csv(f"{OUT_PATH}/cld_features.csv", index=False)

print("Saved:")
print(f"- {OUT_PATH}/cld_features_with_label.csv")
print(f"- {OUT_PATH}/cld_features.csv")

Saved:
- ../data/synthetic/processed/cld_features_with_label.csv
- ../data/synthetic/processed/cld_features.csv


## 11) Close the database connection

In [25]:
conn.close()
print("DB connection closed.")

DB connection closed.


## Summary

We created a **clone-level ML dataset** from early passage measurements only.

Next notebook:
- **Notebook 03 - Modeling**
    - Train a model to predict stability drop using early features
    - Evaluate regression and/or classification performance
    - Interpret feature importance